In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as mb

#### This file can be replaced with any of the opsim files

In [2]:
opsfilename = 'minion_1016_newsky.db'
opsdb       = db.OpsimDatabase(opsfilename)

simdata = opsdb.query_columns(opsdb.defaultTable, 
                              colnames=['fieldRA', 'fieldDec', 'filter', 'night', 'visitTime', 'airmass', 'filtSkyBrightness', 'expMJD', 'fiveSigmaDepth'], 
                              sqlconstraint=None)

### Putting OpSim columns in arrays to make it easier to work with

In [3]:
ra_array      = np.zeros(len(simdata))
dec_array     = np.zeros(len(simdata))
airmass_array = np.zeros(len(simdata))
time_array    = np.zeros(len(simdata))
night_array   = np.zeros(len(simdata))
sigmadepth    = np.zeros(len(simdata))

# creating empty array for strings for filter names
val           = [''] * len(simdata)
filter_array  = np.array(val)

for i in range(len(simdata)):
    ra_array[i]      = simdata[i][0]
    dec_array[i]     = simdata[i][1]
    filter_array[i]  = simdata[i][2]
    night_array[i]   = simdata[i][3]
    time_array[i]    = simdata[i][7] # time should be expMJD
    airmass_array[i] = simdata[i][5]
    sigmadepth[i]    = simdata[i][8]  # 5 sigma depth

In [4]:
import sncosmo
from   astropy.table    import Table
import numpy            as np
import matplotlib.pylab as plt
from   astropy.io       import ascii

#### Setting the filename of sampled supernova parameters. 
This file is obtained by running SNTable notebook that Rahul has. It has (x0,x1,c,t0,z) and host galaxy information. t0 is the peak magnitude time of each supernova.

In [ ]:
filename = "/global/homes/a/anitab/DC2_run1p1_TransientDocs/Notebooks/Hosted_SN_table.csv"
from pandas import read_csv

data  = read_csv(filename, delim_whitespace=True)
#data = vvv['galaxy_id c mB t0 x0 x1 z random_Hosting rand_host zbin "Unnamed: 0" diskMassStellar morphology/diskHalfLightRadiusArcsec morphology/diskMinorAxisArcsec morphology/positionAngle morphology/spheroidHalfLightRadiusArcsec morphology/spheroidMinorAxisArcsec size_bulge_true size_disk_true totalMassStellar uniqueId raJ2000_gal decJ2000_gal z zbin_gals snid']

# uncomment this to see what columns are available
###vvv.columns 

In [ ]:
data.set_index('snid', inplace=True)

In [ ]:
data.head()

### Sampled SN properties in an array
$m_B = 10.635 - \log_{10}(x0)$

In [ ]:
length  = len(data)
'''
zvals   = np.zeros(length) 
x0vals  = np.zeros(length)
x1vals  = np.zeros(length)
cvals   = np.zeros(length)
mBvals  = np.zeros(length)
t0vals  = np.zeros(length)
smass   = np.zeros(length)
ravals  = np.zeros(length)
decvals = np.zeros(length)
snidvals = []
'''
#for i in range(length):
    #zvals[i]   = data[i].split()[6]  # redshifts
    #x0vals[i]  = data[i].split()[4]  # x0 values
    #x1vals[i]  = data[i].split()[5]  # x1 values
    #cvals[i]   = data[i].split()[1]  # c values
    #mBvals[i]  = data[i].split()[2]  # mB values
    #t0vals[i]  = data[i].split()[3]  # Peak magnitude
    #smass[i]   = data[i].split()[20] # Stellar mass of host galaxy
    #ravals[i]  = data[i].split()[21] # Ra of SN
    #decvals[i] = data[i].split()[22] # Dec of SN
 
zvals   = data.z.values
x0vals  = data.x0.values
x1vals  = data.x1.values
cvals   = data.c.values
mBvals  = data.mB.values
t0vals  = data.t0.values
smass   = data.totalMassStellar.values
ravals  = data.raJ2000_gal.values
decvals = data.decJ2000_gal.values


In [ ]:
snids = data.index.values

In [ ]:
np.max(x0vals)

### Choosing rows of OpSim based on sampled SNe

In [ ]:
import os
import opsimsummary as oss
import healpy as hp
print(oss.__version__)

In [ ]:
from opsimsummary import SynOpSim

I need to make sure angleUnit has 'degrees' attribute

In [ ]:
synopsim  = SynOpSim.fromOpSimDB(opsfilename, opsimversion='lsstv3',
                                angleUnit='degrees', usePointingTree=True)

### Generator
This is a Python generator function to make going through data faster. This goes through the observing file and uses Ra and Dec of sampled SNe to determine which SNe would be visible at which times.

In [ ]:
gen = synopsim.pointingsEnclosing(ravals, decvals, circRadius=0., pointingRadius=1.75, usePointingTree=True)

### Time and filters from OpSim file
This is going through the generator and inserting times of observation and filer names in a list. After running this for loop, if we need to go through the generator again, we need to run the gen = ... line again because the generator has now gone all the way to the end. So, we need ro rerun this line:

gen = synopsim.pointingsEnclosing(ravals, decvals, circRadius=0., pointingRadius=1.75, usePointingTree=True)

In [ ]:
#df = next(gen)

In [ ]:
#np.abs(df.expMJD - data.iloc[0].t0).min

In [ ]:
#x = (df.expMJD - data.iloc[0].t0).values

In [ ]:
# np.where((x > -19.) & (x < 49))[0].size

In [ ]:
# data.iloc[1].name

In [ ]:
#snids = []
#times = []
#for i, df in enumerate(gen):
#    x = (df.expMJD - data.iloc[0].t0).values
#    if np.where((x > -19.) & (x < 49))[0].size > 0 :
#        snids.append(data.iloc[i].name)
#        times.append(df.iloc[x].expMJD)

In [ ]:
timeopsim_list       = []
filteropsim_list     = []
sigmadepthopsim_list = []

for i, df in enumerate(gen):
    
    timeopsim = df['expMJD']
    timeopsim_list.append(timeopsim.values)
    
    filteropsim = df['filter']
    filteropsim_list.append(filteropsim.values)
    
    sigmadepthopsim = df['fiveSigmaDepth']
    sigmadepthopsim_list.append(sigmadepthopsim.values)
    
    #raopsim = df['fieldRA']
    #raopsim_list.append(raopsim.values)
    ###raopsim_list += raopsim.values.tolist() # this is to make the 2D array into 1D array. Not needed in general though

### Plotting SNe and their corresponding OpSim points
This shows supernovae as red dots and the OpSim visits as blue dots. There might be multiple visits at the same location so some points are overlapping and they are for different times.

In [ ]:
'''
ifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)
ify      = np.array([1,6,11,16,21])   # subplot numbers (for turning y ticks on/off)

plt.figure(1, figsize=(15,15))

for i in range(25):
    
    plt.subplot(5,5,i+1)
    plt.plot(np.rad2deg(decopsim_list[1000*i]), np.rad2deg(raopsim_list[1000*i]), ".", color="blue")
    plt.plot(decvals[1000*i], ravals[1000*i], ".", color="red")
    plt.ylim(50,60)
    plt.xlim(-34,-24)
    plt.tick_params(axis='both', labelleft='off', labelbottom='off')

    if i+1 in ifx:
        plt.xlabel("Ra [deg]",fontsize=15)
        plt.tick_params(axis='x', labelbottom='on')
    if i+1 in ify:
        plt.ylabel("Dec [deg]",fontsize=15)
        plt.tick_params(axis='y', labelleft='on')
    
plt.suptitle("SNe in red with opsim visits in blue")
'''

### Checking time range of OpSim with $t_0$
Looping over all supernova and checking the observing times corresponding to supernova location from OpSim file and adding the ones that are within (t0 - 19 < OpSimtime < t0+49 ) range. This is the SALT model time range. If the OpSim time is not within this time range, the flux calculated would be 0 and we do not get any useful information. So, I want to reduce the number of data and only work with the useful data.

In [ ]:
from lsst.sims.catUtils.supernovae import SNObject
from lsst.sims.photUtils.BandpassDict import BandpassDict

# Setting the LSST BandPass object to be used in BandFlux function
LSST_BandPass = BandpassDict.loadTotalBandpassesFromFiles()

mlist     = []
jlist_tot = []
snids_tot = []

for m in range(len(timeopsim_list)):
    jlist = []
    for j in range(len(timeopsim_list[m])):
        if (timeopsim_list[m][j] > t0vals[m] -19 and timeopsim_list[m][j] < t0vals[m] + 49):
            jlist.append(j)
    if jlist:
        mlist.append(m)
        jlist_tot.append(jlist)
        

In [ ]:
snids[mlist]

In [ ]:
data.loc[snids].query('z < 0.1').z.size

In [ ]:
data.loc[snids, 'z']

In [ ]:
for times in timeopsim_list:
    if 0 == len(times):
        continue
    

### Calculating the flux
Goes through a loop for each supernova and sets the parameters for each and then calculates the flux for all the visits

In [ ]:
fluxlist_tot    = [] 
timelist_tot    = []
fluxerrlist_tot = []
filterlist_tot  = []

# Going through each SN
for SN_id, val in enumerate(mlist[:400]):
    # setting parameters for each supernova
    params   = {'x0': x0vals[val], 'x1': x1vals[val], 'c': cvals[val], 't0':t0vals[val], 'z':zvals[val]}
    snobject = SNObject(ravals[val], decvals[val])
    snobject.set(**params)
    snobject.set_MWebv(0.0) # Finally we should not have this
    
    # to make filternames what the code accepts using list comprehension to make it faster
    filters_modified = [LSST_BandPass[key] for key in filteropsim_list[val]]

    fluxlist    = []
    timelist    = []
    fluxerrlist = []
    filterlist  = []
    #snid_useful = []
    
    # going through all the visits for each single supernova
    for j in jlist_tot[SN_id]:
        fluxval = snobject.catsimBandFlux(bandpassobject = filters_modified[j], time = timeopsim_list[val][j])
        fluxlist.append(fluxval)
        timelist.append(timeopsim_list[val][j]) # appending the times of observation to a list
        
        fluxvalerr = snobject.catsimBandFluxError(bandpassobject = filters_modified[j], time = timeopsim_list[val][j], m5 = sigmadepthopsim_list[val][j])
        fluxerrlist.append(fluxvalerr)
        
        filterlist.append(filteropsim_list[val][j])
    
    fluxlist_tot.append(fluxlist) # shape should be (number of SN, entries for each SN)
    timelist_tot.append(timelist) # shape should be (number of SN, entries for each SN)
    fluxerrlist_tot.append(fluxerrlist) # shape should be (number of SN, entries for each SN)
    filterlist_tot.append(filterlist)

### Plotting flux vs. time
This is the fluxes obtained using catsimBandFlux() function for a selection of SNe vs expMJD time. The points are fluxes in all bands so that is why they have funny and oscillating shapes.

In [ ]:
ifx      = np.array([21,22,23,24,25]) # subplot numbers (for turning x ticks on/off)
ify      = np.array([1,6,11,16, 21])

plt.figure(1, figsize=(15,15))

for i in range(25):
    plt.subplot(5,5,i+1)
    
    indx = np.arange(0,len(timelist_tot[i]))
    Z = [x for _,x in sorted(zip(timelist_tot[i],indx))]
        
    plt.errorbar(np.sort(timelist_tot[i]), np.array(fluxlist_tot[i])[Z], yerr = np.array(fluxerrlist_tot[i])[Z], color='blue')
    
    #plt.plot(np.sort(timelist_tot[i]), np.array(fluxlist_tot[i])[Z], "o", color='blue')
    #plt.plot(np.sort(timelist_tot[i]), np.array(fluxlist_tot[i])[Z], color='blue')
    
    #plt.tick_params(axis='both', labelleft='off', labelbottom='off')

    if i+1 in ifx:
        plt.xlabel("Time [MJD]",fontsize=15)
    #    plt.tick_params(axis='x', labelbottom='on')
    if i+1 in ify:
        plt.ylabel("Flux",fontsize=15)
    #    plt.tick_params(axis='y', labelleft='on')

### sncosmo fitting
https://sncosmo.readthedocs.io/en/v1.6.x/photdata.html?highlight=Photometric%20Data

I want to make a table that has all the information to pass to sncosmo. It should be a table that includes:
(time, band, flux, fluxerr, zp, zpsys). 

The band should be in the form of 'lsstu', 'lsstg', ...

zp: 'ab'

### Creating a table to pass to sncosmo
This function creates a table that has (time, band, flux, fluxerr, zp, zpsys) values in a Table that can be passed to sncosmo fit_lc() function. The format of the bands should be correct as it is done here.

In [ ]:
from astropy.table import Table, Column

modelsncosmo     = sncosmo.Model(source = 'salt2-extended') # setting SN model

def create_Table(SnNum, timelist=timelist_tot):
    t            = Table()
    
    length       = len(timelist[SnNum])  # length of time and flux array for each SN
    zpsysarray   = np.array(['ab'] * length) # creating an array of zpsys

    t['time']    = timelist_tot[SnNum]       # setting time
    # This line is to make bands in a format readable for sncosmo
    t['band']    = np.array(['lsst' + np.array(filterlist_tot[SnNum])[i] for i in range(len(filterlist_tot[SnNum]))]) #('lsst' + filterlist_tot[indxval])
    t['flux']    = fluxlist_tot[SnNum]       # flux
    t['fluxerr'] = fluxerrlist_tot[SnNum]    # fluxerr
    t['zp']      = np.zeros(length)          # zp (all zp values are 0)
    t['zpsys']   = zpsysarray                # zpsys

    t['band']    = t['band'].astype(str)     # to make sure band has the correct format
    
    return t

In [ ]:
from analyzeSN import LightCurve

In [ ]:
t  = create_Table(260)

In [ ]:
snids[mlist[260]]

In [ ]:
#lc = LightCurve(t.to_pandas().query('band != "sdssy"').sort_values(by='time'))

In [ ]:
#t  = Table(np.sort(t))que

In [ ]:
modelsncosmo.set(z=data.loc[snids[mlist[260]]].z)
modelsncosmo.set(t0=data.loc[snids[mlist[260]]].t0)
result, fitted_model  = sncosmo.fit_lc(np.sort(t), modelsncosmo, ['x0', 'x1', 'c'])

In [ ]:
modelsncosmo.set(z=data.loc[mlist[260]].z)
modelsncosmo.set(t0=data.loc[mlist[260]].t0)
result, fitted_model  = sncosmo.fit_lc(lc.snCosmoLC(), modelsncosmo, ['x0', 'x1', 'c'])
#result, fitted_model  = sncosmo.fit_lc(lc.snCosmoLC(), modelsncosmo, ['t0', 'x0', 'x1', 'c'])
                                      #bounds=dict(z=(0., 1.0)))

In [ ]:
from collections import OrderedDict as Odict

In [ ]:
params = Odict(data.loc[snids[mlist[260]]][['z', 'c', 'x1', 'x0', 't0']])

In [ ]:
!echo ${THROUGHPUTS_DIR}

In [ ]:
import analyzeSN

In [ ]:
truth = sncosmo.Model(source='salt2-extended')
truth.set(**params)

In [ ]:
truth_salt = sncosmo.Model(source='salt2')
truth_salt.set(**params)

In [ ]:
print(truth_salt)

In [ ]:
sncosmo.get_bandpass('lsstr').wave

In [ ]:
t

In [ ]:
snobject

In [ ]:
snobject.SNstate

In [ ]:
fig = sncosmo.plot_lc(np.sort(t), model=(fitted_model, truth), errors=result.errors, 
                    model_label=("fit", "truth"), pulls=True)

In [ ]:
result

In [ ]:
def calc_lc(data, zmin, zmax):
    #band_indx = np.where(data['band'] == band) # finding indices of table with the specified band
    result, fitted_model  = sncosmo.fit_lc(np.sort(data), modelsncosmo, ['z', 't0', 'x0', 'x1', 'c'], bounds={'z':(zmin, zmax)})
    sncosmo.plot_lc(np.sort(data[band_indx]), model=fitted_model, errors=result.errors)
    return result, fitted_model

In [ ]:
#sys.path.insert(0,"/global/homes/a/anitab/.local/cori/")

In [ ]:
#import sncosmo

In [ ]:
#val1, val2 = calc_lc(t, 0.3, 0.7)

In [ ]:
#val1, val2 = calc_lc(t, 'lssty', 0.3, 0.7)

To find which SNe have larger number of times so that we have more points to fit for the fluxes in each band

In [ ]:
for i in range(500):
    if np.shape(timelist_tot[i])[0] > 1300:
        print (i)

In [ ]:
len(timelist_tot[260])

### LSST code example that works!

In [ ]:
bandpassnames = ['u', 'g', 'r', 'i', 'z', 'y']
from lsst.sims.photUtils.BandpassDict import BandpassDict
LSST_BandPass = BandpassDict.loadTotalBandpassesFromFiles()
SN = SNObject(ra=30., dec=-60.)
SN.set(z=0.96, t0=571181, x1=2.66, c=0.353, x0=1.796112e-06)
SN.catsimBandFlux(bandpassobject=LSST_BandPass['r'], time=571190.)

### $m_B$ obtained from $x_0$

In [ ]:
def mB(x0):
    return 10.635 - (2.5 * np.log10(x0))